In [ ]:
# Mengimpor library PyTorch
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

# Mengimpor library HuggingFace Transformers
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoConfig

# Mengimpor library lainnya
import pandas as pd
import numpy as np
import random

# Mendefinisikan fungsi untuk memisahkan teks dan label menjadi list of tokens
def tokenize_text_and_label(text, label):
  # Menggunakan tokenizer dari IndoBERT
  tokenizer = AutoTokenizer.from_pretrained("indolem/indobert-base-uncased")

  # Melakukan tokenisasi teks dan label
  text_tokens = tokenizer.tokenize(text)
  label_tokens = label.split()

  # Menyesuaikan panjang label_tokens dengan text_tokens
  # Karena tokenizer BERT menggunakan WordPiece, beberapa kata dapat terpecah menjadi beberapa subword
  # Misalnya, kata "Pengalaman" menjadi ["peng", "##ala", "##man"]
  # Untuk kasus ini, kita hanya memberikan label pada subword pertama dan memberikan label "X" pada subword lainnya
  # Label "X" akan diabaikan saat melatih model
  new_label_tokens = []
  i = 0
  for token in text_tokens:
    if token.startswith("##"):
      new_label_tokens.append("X")
    else:
      new_label_tokens.append(label_tokens[i])
      i += 1

  # Mengembalikan text_tokens dan new_label_tokens sebagai output
  return text_tokens, new_label_tokens

# Mendefinisikan fungsi untuk mengubah list of tokens menjadi list of ids
def tokens_to_ids(text_tokens, label_tokens):
  # Menggunakan tokenizer dan config dari IndoBERT
  tokenizer = AutoTokenizer.from_pretrained("indolem/indobert-base-uncased")
  config = AutoConfig.from_pretrained("indolem/indobert-base-uncased")

  # Mengubah text_tokens menjadi input_ids dan attention_mask
  # Input_ids adalah representasi numerik dari text_tokens yang dapat dimengerti oleh model BERT
  # Attention_mask adalah tensor yang menunjukkan posisi token yang sebenarnya (1) dan yang dipad (0)
  input_ids = tokenizer.convert_tokens_to_ids(text_tokens)
  attention_mask = [1] * len(input_ids)

  # Mengubah label_tokens menjadi label_ids
  # Label_ids adalah representasi numerik dari label_tokens yang dapat dimengerti oleh model BERT
  # Kita menggunakan dictionary yang telah disediakan oleh config dari IndoBERT
  label_map = config.label2id
  label_ids = [label_map[label] for label in label_tokens]

  # Mengembalikan input_ids, attention_mask, dan label_ids sebagai output
  return input_ids, attention_mask, label_ids

# Mendefinisikan kelas JobDataset yang merupakan subclass dari kelas Dataset PyTorch
class JobDataset(Dataset):
  
    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        # Mengambil data pada indeks tertentu
        row = self.data.iloc[index]

        # Memisahkan teks dan label menjadi list of tokens
        text = row["text"]
        label = row["label"]
        text_tokens, label_tokens = tokenize_text_and_label(text,label)

        # Mengubah list of tokens menjadi list of ids
        input_ids, attention_mask, label_ids = tokens_to_ids(text_tokens,label_tokens)

        # Mengubah list of ids menjadi tensor PyTorch
        input_ids = torch.tensor(input_ids)
        attention_mask = torch.tensor(attention_mask)
        label_ids = torch.tensor(label_ids)

        # Mengembalikan sebuah dictionary yang berisi input_ids, attention_mask, dan label_ids sebagai output
        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "label_ids": label_ids
        }

# Mendefinisikan kelas JobDataLoader yang merupakan subclass dari kelas DataLoader PyTorch
class JobDataLoader(DataLoader):

    def __init__(self, dataset, batch_size):
        self.dataset = dataset
        self.batch_size = batch_size
    
    def __iter__(self):
        # Membuat list of indices dari dataset
        indices = list(range(len(self.dataset)))

        # Melakukan shuffling pada list of indices
        random.shuffle(indices)

        # Membuat list kosong untuk menyimpan batch data
        batch_data = []

        # Melakukan iterasi pada setiap indeks dalam list of indices
        for i in indices:
            # Mengambil data pada indeks tertentu dari dataset dengan menggunakan method __getitem__
            data = self.dataset[i]

            # Menambahkan data ke list batch data
            batch_data.append(data)

            # Jika panjang list batch data sudah sama dengan batch size atau sudah mencapai indeks terakhir, maka proses batch data
            if len(batch_data) == self.batch_size or i == indices[-1]:
                # Mengambil input_ids, attention_mask, dan label_ids dari setiap data dalam batch data
                input_ids = [data["input_ids"] for data in batch_data]
                attention_mask = [data["attention_mask"] for data in batch_data]
                label_ids = [data["label_ids"] for data in batch_data]

                # Melakukan padding pada input_ids, attention_mask, dan label_ids agar memiliki panjang yang sama dalam satu batch
                # Menggunakan nilai 0 untuk padding input_ids dan attention_mask, dan nilai -100 untuk padding label_ids
                # Nilai -100 akan diabaikan oleh loss function saat melatih model
                input_ids = pad_sequence(input_ids, batch_first=True, padding_value=0)
                attention_mask = pad_sequence(attention_mask, batch_first=True, padding_value=0)
                label_ids = pad_sequence(label_ids, batch_first=True, padding_value=-100)

                # Mengembalikan sebuah dictionary yang berisi input_ids, attention_mask, dan label_ids sebagai output dari iterator
                yield {
                    "input_ids": input_ids,
                    "attention_mask": attention_mask,
                    "label_ids": label_ids
                }

                # Mengosongkan list batch data untuk batch selanjutnya
                batch_data = []
